# 🔬 HBV 그룹 비교 분석 (Real Data - Complete)

**목적**: HBV Stage별 면역세포 분포 비교 (GSE182159 실제 데이터)

---

## 📊 HBV Stages

| Stage | 설명 |
|-------|------|
| IT | Immune Tolerant (면역 관용기) |
| IA | Immune Active (면역 활성기) |
| IC | Immune Control (면역 조절기) |
| GZ | Gray Zone |
| Resolved | 완치 |

## 📋 분석 단계

| STEP | 내용 |
|------|------|
| 1 | 환경 설정 및 데이터 로드 |
| 2 | 그룹별 세포 분포 비교 (UMAP) |
| 3 | 세포 유형별 비율 분석 |
| 4 | 조직별 (Blood vs Liver) 분석 |
| 5 | 탈진 마커 발현 분석 |
| 6 | 기본 통계 검정 |
| 7 | **상세 통계 분석 및 수치 테이블** |
| 8 | 결과 저장 |

⚠️ **사전 조건**: `Step1_HBV_real.ipynb` 실행 완료

In [ ]:
# @title STEP 1: 환경 설정 및 데이터 로드
import os, sys
from google.colab import drive
drive.mount('/content/drive')

PROJECT_PATH = '/content/drive/MyDrive/tahoe-x1'
ANALYSIS_PATH = f'{PROJECT_PATH}/viral_hepatitis/analysis_results'
OUTPUT_PATH = f'{PROJECT_PATH}/viral_hepatitis/comparison_results'
os.makedirs(OUTPUT_PATH, exist_ok=True)

# 패키지 설치
!pip install scanpy anndata scipy statsmodels seaborn mygene -q

import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.stats.multitest import multipletests
import scipy.sparse as sp
import itertools
import gc

# Step 1 결과 로드
print('📊 분석 결과 검색 중...')
h5ad_files = [f for f in os.listdir(ANALYSIS_PATH) if f.startswith('hbv_real_tahoe_analysis_full')]

if not h5ad_files:
    h5ad_files = [f for f in os.listdir(ANALYSIS_PATH) if f.endswith('.h5ad')]

print(f'   발견된 파일: {h5ad_files}')
latest_file = sorted(h5ad_files)[-1]
print(f'   로드할 파일: {latest_file}')

print('\n📊 데이터 로딩 중...')
adata = sc.read_h5ad(f'{ANALYSIS_PATH}/{latest_file}')

# HBV Stage 컬럼 확인
stage_col = None
for col in ['hbv_stage', 'Stage', 'stage']:
    if col in adata.obs.columns:
        stage_col = col
        break

print(f'\n✅ STEP 1 완료!')
print(f'   세포 수: {adata.n_obs:,}')
print(f'   유전자 수: {adata.n_vars:,}')
print(f'   Stage 컬럼: {stage_col}')

if stage_col:
    print(f'\n📊 HBV Stage 분포:')
    print(adata.obs[stage_col].value_counts())

print(f'\n📊 조직별 분포:')
print(adata.obs['tissue'].value_counts())

In [ ]:
# @title STEP 2: 그룹별 세포 분포 비교 (UMAP)
print('📊 그룹별 UMAP 시각화')

# 전체 UMAP
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

sc.pl.umap(adata, color='cell_type', ax=axes[0], show=False,
           title='Cell Types', legend_loc='on data', legend_fontsize=8)

sc.pl.umap(adata, color=stage_col, ax=axes[1], show=False,
           title='HBV Stage')

sc.pl.umap(adata, color='tissue', ax=axes[2], show=False,
           title='Tissue (Blood vs Liver)')

plt.tight_layout()
plt.savefig(f'{OUTPUT_PATH}/step2_umap_overview.png', dpi=150, bbox_inches='tight')
plt.show()

# Stage별 개별 UMAP
stages = sorted(adata.obs[stage_col].dropna().unique())
n_stages = len(stages)
print(f'\n📊 Stage별 UMAP ({n_stages}개 그룹)')

cols = min(4, n_stages)
rows = (n_stages + cols - 1) // cols
fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 5*rows))
axes = np.array(axes).flatten() if n_stages > 1 else [axes]

for i, stage in enumerate(stages):
    if i < len(axes):
        adata_subset = adata[adata.obs[stage_col] == stage]
        sc.pl.umap(adata_subset, color='cell_type', ax=axes[i], show=False,
                   title=f'{stage} (n={len(adata_subset):,})', legend_loc='none')

for j in range(i+1, len(axes)):
    axes[j].set_visible(False)

plt.tight_layout()
plt.savefig(f'{OUTPUT_PATH}/step2_umap_by_stage.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\n✅ STEP 2 완료!')

In [ ]:
# @title STEP 3: 세포 유형별 비율 분석
print('📊 세포 유형별 비율 분석')

# 그룹별 세포 유형 비율 계산
cell_type_counts = adata.obs.groupby([stage_col, 'cell_type']).size().unstack(fill_value=0)
cell_type_ratios = cell_type_counts.div(cell_type_counts.sum(axis=1), axis=0) * 100

print('\n📋 그룹별 세포 유형 비율 (%):')
print(cell_type_ratios.round(1))

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 1. Stacked bar
cell_type_ratios.plot(kind='bar', stacked=True, ax=axes[0], colormap='tab10')
axes[0].set_title('Cell Type Distribution by HBV Stage', fontsize=12)
axes[0].set_xlabel('HBV Stage')
axes[0].set_ylabel('Percentage (%)')
axes[0].legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=8)
axes[0].tick_params(axis='x', rotation=45)

# 2. Heatmap
sns.heatmap(cell_type_ratios.T, annot=True, fmt='.1f', cmap='YlOrRd', ax=axes[1],
            cbar_kws={'label': 'Percentage (%)'})
axes[1].set_title('Cell Type Percentage Heatmap', fontsize=12)
axes[1].set_xlabel('HBV Stage')
axes[1].set_ylabel('Cell Type')

plt.tight_layout()
plt.savefig(f'{OUTPUT_PATH}/step3_cell_type_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\n✅ STEP 3 완료!')

In [ ]:
# @title STEP 4: 조직별 분석 (Blood vs Liver)
print('📊 조직별 세포 분포 분석 (Blood vs Liver)')

# 조직별 세포 유형 분포
for tissue in ['Blood', 'Liver']:
    adata_tissue = adata[adata.obs['tissue'] == tissue]
    print(f'\n📊 {tissue} ({adata_tissue.n_obs:,} cells):')
    
    tissue_counts = adata_tissue.obs.groupby([stage_col, 'cell_type']).size().unstack(fill_value=0)
    tissue_ratios = tissue_counts.div(tissue_counts.sum(axis=1), axis=0) * 100
    print(tissue_ratios.round(1))

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

for idx, tissue in enumerate(['Blood', 'Liver']):
    adata_tissue = adata[adata.obs['tissue'] == tissue]
    tissue_counts = adata_tissue.obs.groupby([stage_col, 'cell_type']).size().unstack(fill_value=0)
    tissue_ratios = tissue_counts.div(tissue_counts.sum(axis=1), axis=0) * 100
    
    tissue_ratios.plot(kind='bar', stacked=True, ax=axes[idx], colormap='tab10')
    axes[idx].set_title(f'{tissue} - Cell Type by HBV Stage', fontsize=12)
    axes[idx].set_xlabel('HBV Stage')
    axes[idx].set_ylabel('Percentage (%)')
    axes[idx].tick_params(axis='x', rotation=45)
    if idx == 1:
        axes[idx].legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=8)
    else:
        axes[idx].legend().set_visible(False)

plt.tight_layout()
plt.savefig(f'{OUTPUT_PATH}/step4_tissue_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\n✅ STEP 4 완료!')

In [ ]:
# @title STEP 5: 탈진 마커 발현 분석
import mygene

print('🔬 T세포 탈진 마커 분석')

# 마커 정의
markers_symbol = {
    'Exhaustion': ['PDCD1', 'HAVCR2', 'LAG3', 'CTLA4', 'TIGIT'],
    'Treg': ['FOXP3', 'IL2RA', 'IKZF2'],
    'Cytotoxic': ['GZMB', 'PRF1', 'IFNG', 'GNLY'],
    'Activation': ['CD69', 'HLA-DRA', 'CD38']
}

# Gene Symbol → Ensembl ID 변환
all_markers = []
for genes in markers_symbol.values():
    all_markers.extend(genes)

mg = mygene.MyGeneInfo()
results = mg.querymany(all_markers, scopes='symbol', fields='ensembl.gene', species='human', verbose=False)

symbol_to_ensembl = {}
for r in results:
    if 'ensembl' in r:
        ensembl = r['ensembl']
        if isinstance(ensembl, list):
            ensembl = ensembl[0]
        if isinstance(ensembl, dict) and 'gene' in ensembl:
            symbol_to_ensembl[r['query']] = ensembl['gene']

print(f'   Gene Symbol → Ensembl 변환: {len(symbol_to_ensembl)}/{len(all_markers)}')

# 사용 가능한 마커
available_markers = {}
for category, genes in markers_symbol.items():
    ensembl_genes = [symbol_to_ensembl.get(g) for g in genes if symbol_to_ensembl.get(g) in adata.var_names]
    symbols = [g for g in genes if symbol_to_ensembl.get(g) in adata.var_names]
    if ensembl_genes:
        available_markers[category] = {'ensembl': ensembl_genes, 'symbol': symbols}
        print(f'   {category}: {symbols}')

# 탈진 마커 시각화
if 'Exhaustion' in available_markers:
    print('\n📊 탈진 마커 Stage별 발현')
    
    n_markers = len(available_markers['Exhaustion']['ensembl'])
    fig, axes = plt.subplots(1, n_markers, figsize=(4*n_markers, 5))
    if n_markers == 1:
        axes = [axes]
    
    for ax, (ensembl, symbol) in zip(axes, zip(available_markers['Exhaustion']['ensembl'], 
                                              available_markers['Exhaustion']['symbol'])):
        sc.pl.violin(adata, ensembl, groupby=stage_col, ax=ax, show=False)
        ax.set_title(f'{symbol}')
        ax.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig(f'{OUTPUT_PATH}/step5_exhaustion_markers.png', dpi=150, bbox_inches='tight')
    plt.show()

print(f'\n✅ STEP 5 완료!')

In [ ]:
# @title STEP 6: 기본 통계 검정
print('📊 기본 통계적 유의성 검정')

# Chi-square test (전체)
print('\n1️⃣ Chi-square test (세포 유형 × HBV Stage):')
contingency_table = adata.obs.groupby([stage_col, 'cell_type']).size().unstack(fill_value=0)
chi2_overall, p_value_overall, dof, expected = stats.chi2_contingency(contingency_table)
print(f'   Chi-square: {chi2_overall:.2f}')
print(f'   Degrees of freedom: {dof}')
print(f'   p-value: {p_value_overall:.2e}')
print(f'   결과: {"✅ 유의함 (p < 0.05)" if p_value_overall < 0.05 else "❌ 유의하지 않음"}')

# 조직별 Chi-square
print('\n2️⃣ 조직별 Chi-square test:')
for tissue in ['Blood', 'Liver']:
    adata_tissue = adata[adata.obs['tissue'] == tissue]
    cont_table = adata_tissue.obs.groupby([stage_col, 'cell_type']).size().unstack(fill_value=0)
    chi2_t, p_t, _, _ = stats.chi2_contingency(cont_table)
    print(f'   {tissue}: Chi2={chi2_t:.2f}, p={p_t:.2e} {"✅" if p_t < 0.05 else ""}')

print(f'\n✅ STEP 6 완료!')

In [ ]:
# @title STEP 7: 상세 통계 분석 및 수치 테이블
print('📊 상세 통계 분석')

stages = sorted(adata.obs[stage_col].dropna().unique())
cell_types = sorted(adata.obs['cell_type'].unique())

# ==========================================
# 7-1. 세포 유형 × Stage 비율 테이블 (수치)
# ==========================================
print('\n' + '='*70)
print('📋 7-1. 세포 유형별 비율 테이블 (%) - Stage별')
print('='*70)

ratio_table = pd.DataFrame(index=cell_types, columns=stages)
count_table = pd.DataFrame(index=cell_types, columns=stages)

for stage in stages:
    stage_cells = adata[adata.obs[stage_col] == stage]
    total = len(stage_cells)
    for ct in cell_types:
        count = (stage_cells.obs['cell_type'] == ct).sum()
        count_table.loc[ct, stage] = count
        ratio_table.loc[ct, stage] = round(count / total * 100, 2)

print('\n[세포 수]')
print(count_table.to_string())
print('\n[비율 (%)]')
print(ratio_table.to_string())

count_table.to_csv(f'{OUTPUT_PATH}/detailed_cell_counts.csv')
ratio_table.to_csv(f'{OUTPUT_PATH}/detailed_cell_ratios.csv')

# ==========================================
# 7-2. Pairwise Chi-square 검정
# ==========================================
print('\n' + '='*70)
print('📋 7-2. Pairwise Chi-square 검정 (Stage 쌍별 비교)')
print('='*70)

pairwise_results = []

for stage1, stage2 in itertools.combinations(stages, 2):
    subset = adata[adata.obs[stage_col].isin([stage1, stage2])]
    cont_table = subset.obs.groupby([stage_col, 'cell_type']).size().unstack(fill_value=0)
    
    if len(cont_table) == 2:
        chi2, p_val, dof, _ = stats.chi2_contingency(cont_table)
        pairwise_results.append({
            'Comparison': f'{stage1} vs {stage2}',
            'Chi2': round(chi2, 2),
            'p-value': p_val,
            'Significant (p<0.05)': '✅' if p_val < 0.05 else ''
        })

pairwise_df = pd.DataFrame(pairwise_results)

# FDR 보정
if len(pairwise_df) > 0:
    _, p_adj, _, _ = multipletests(pairwise_df['p-value'], method='fdr_bh')
    pairwise_df['p-adjusted (FDR)'] = p_adj
    pairwise_df['FDR Significant'] = ['✅' if p < 0.05 else '' for p in p_adj]

print(pairwise_df.to_string(index=False))
pairwise_df.to_csv(f'{OUTPUT_PATH}/pairwise_chi2_results.csv', index=False)

# ==========================================
# 7-3. 세포 유형별 Stage 간 유의성 검정
# ==========================================
print('\n' + '='*70)
print('📋 7-3. 세포 유형별 Stage 간 비율 차이 검정')
print('='*70)

celltype_stats = []

for ct in cell_types:
    row = {'Cell Type': ct}
    
    proportions = []
    for stage in stages:
        stage_cells = adata[adata.obs[stage_col] == stage]
        prop = (stage_cells.obs['cell_type'] == ct).mean()
        row[f'{stage} (%)'] = round(prop * 100, 2)
        proportions.append(prop)
    
    row['Max-Min Diff'] = round((max(proportions) - min(proportions)) * 100, 2)
    
    ct_binary = (adata.obs['cell_type'] == ct).astype(int)
    cont = pd.crosstab(adata.obs[stage_col], ct_binary)
    if cont.shape[1] == 2:
        chi2, p_val, _, _ = stats.chi2_contingency(cont)
        row['Chi2'] = round(chi2, 2)
        row['p-value'] = f'{p_val:.2e}'
        row['Sig'] = '✅' if p_val < 0.05 else ''
    
    celltype_stats.append(row)

celltype_df = pd.DataFrame(celltype_stats)
print(celltype_df.to_string(index=False))
celltype_df.to_csv(f'{OUTPUT_PATH}/celltype_stage_statistics.csv', index=False)

# ==========================================
# 7-4. 조직별 (Blood vs Liver) 비교
# ==========================================
print('\n' + '='*70)
print('📋 7-4. 조직별 세포 분포 비교 (Blood vs Liver)')
print('='*70)

tissue_comparison = []

for ct in cell_types:
    blood_cells = adata[adata.obs['tissue'] == 'Blood']
    liver_cells = adata[adata.obs['tissue'] == 'Liver']
    
    blood_ratio = (blood_cells.obs['cell_type'] == ct).mean() * 100
    liver_ratio = (liver_cells.obs['cell_type'] == ct).mean() * 100
    
    blood_ct = (blood_cells.obs['cell_type'] == ct).sum()
    blood_other = len(blood_cells) - blood_ct
    liver_ct = (liver_cells.obs['cell_type'] == ct).sum()
    liver_other = len(liver_cells) - liver_ct
    
    _, p_val = stats.fisher_exact([[blood_ct, blood_other], [liver_ct, liver_other]])
    
    tissue_comparison.append({
        'Cell Type': ct,
        'Blood (%)': round(blood_ratio, 2),
        'Liver (%)': round(liver_ratio, 2),
        'Difference': round(blood_ratio - liver_ratio, 2),
        'p-value': f'{p_val:.2e}',
        'Sig': '✅' if p_val < 0.05 else ''
    })

tissue_df = pd.DataFrame(tissue_comparison)
print(tissue_df.to_string(index=False))
tissue_df.to_csv(f'{OUTPUT_PATH}/tissue_comparison_statistics.csv', index=False)

print(f'\n✅ STEP 7 완료!')

In [ ]:
# @title STEP 8: 종합 결과 저장
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# 유의한 비교 수 계산
n_sig_pairwise = (pairwise_df['Significant (p<0.05)'] == '✅').sum() if len(pairwise_df) > 0 else 0
n_sig_fdr = (pairwise_df['FDR Significant'] == '✅').sum() if 'FDR Significant' in pairwise_df.columns else 0
n_sig_tissue = (tissue_df['Sig'] == '✅').sum()
n_sig_celltype = (celltype_df['Sig'] == '✅').sum()

# Summary 생성
summary = f"""
================================================================================
🦠 HBV 면역세포 비교 분석 결과 (GSE182159 Real Data)
================================================================================

📅 분석 일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

================================================================================
📊 데이터 요약
================================================================================
   - 총 세포 수: {adata.n_obs:,}
   - 유전자 수: {adata.n_vars:,}
   - HBV Stage 수: {len(stages)}
   - 세포 유형 수: {len(cell_types)}

📋 조직별 세포 수:
{adata.obs['tissue'].value_counts().to_string()}

📋 HBV Stage별 세포 수:
{adata.obs[stage_col].value_counts().to_string()}

================================================================================
🔬 통계 분석 결과 요약
================================================================================

1️⃣ 전체 Chi-square (Cell Type × Stage):
   - Chi-square: {chi2_overall:.2f}
   - p-value: {p_value_overall:.2e}
   - 결론: {'세포 분포가 HBV Stage에 따라 유의하게 다름 ✅' if p_value_overall < 0.05 else '유의한 차이 없음'}

2️⃣ Pairwise Stage 비교:
   - 총 비교 쌍: {len(pairwise_df)}
   - 유의한 쌍 (p<0.05): {n_sig_pairwise}
   - FDR 보정 후 유의: {n_sig_fdr}

3️⃣ 세포 유형별 Stage 간 차이:
   - 유의한 세포 유형: {n_sig_celltype} / {len(cell_types)}

4️⃣ Blood vs Liver 비교:
   - 유의한 세포 유형: {n_sig_tissue} / {len(cell_types)}

================================================================================
📁 저장된 파일
================================================================================
시각화:
   - step2_umap_overview.png
   - step2_umap_by_stage.png
   - step3_cell_type_distribution.png
   - step4_tissue_comparison.png
   - step5_exhaustion_markers.png

통계 테이블:
   - detailed_cell_counts.csv (세포 수)
   - detailed_cell_ratios.csv (비율 %)
   - pairwise_chi2_results.csv (Stage 쌍별 비교)
   - celltype_stage_statistics.csv (세포 유형별 검정)
   - tissue_comparison_statistics.csv (Blood vs Liver)

================================================================================
"""

print(summary)

# Summary 저장
with open(f'{OUTPUT_PATH}/analysis_summary_{timestamp}.txt', 'w', encoding='utf-8') as f:
    f.write(summary)

print('\n🎉 모든 분석이 완료되었습니다!')
print(f'📁 결과 폴더: {OUTPUT_PATH}')
print(f'\n📋 생성된 CSV 파일:')
for f in os.listdir(OUTPUT_PATH):
    if f.endswith('.csv'):
        print(f'   - {f}')

---
## 📚 결과 해석 가이드

### HBV Stage별 예상 면역 특성:

| Stage | CD8+ T | NK cells | Exhaustion | 설명 |
|-------|--------|----------|------------|------|
| IT (Immune Tolerant) | 낮음 | 낮음 | 높음 | 면역 관용, 바이러스 활발 복제 |
| IA (Immune Active) | 높음 | 높음 | 중간 | 면역 활성화, 간 손상 |
| IC (Immune Control) | 중간 | 중간 | 낮음 | 바이러스 억제, 안정기 |
| Resolved | 정상 | 정상 | 낮음 | 완치 상태 |

### 통계적 해석:
- **p < 0.05**: 통계적으로 유의한 차이
- **FDR 보정**: 다중 비교 시 위양성 감소
- **Max-Min Diff**: Stage 간 최대 비율 차이

### 다음 분석 제안:
1. T세포 아형 세분화 분석
2. 간 상주 면역세포 vs 순환 면역세포 비교
3. 치료 반응 예측 바이오마커 발굴